In [91]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf


In [6]:
from keras.layers import Input, Dense, Reshape,multiply,Dropout,Concatenate,UpSampling2D, Conv2D, LeakyReLU, Embedding, Flatten
from keras.models import Model,Sequential
from keras.optimizers import Adam


In [24]:


def define_generator(latent_dim):
    model = Sequential()
    model.add(Dense(128 * 7 * 7, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    
    model.add(Reshape((7, 7, 128)))
    
    model.add(UpSampling2D())
    model.add(LeakyReLU(alpha=0.2))
    
    model.add(UpSampling2D())
    model.add(LeakyReLU(alpha=0.2))
    

    z = Input(shape=(latent_dim,))
    label = Input(shape=(1,), dtype='uint8')
    label_embed = Embedding(10, latent_dim, input_length=1)(label)
    label_embed = Flatten()(label_embed)
    combined = multiply([z, label_embed])

    img = model(combined)

    return Model([z, label], img),model

latent_dim = 100
g_model,model = define_generator(latent_dim)

model.summary()


Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 6272)              633472    
                                                                 
 leaky_re_lu_22 (LeakyReLU)  (None, 6272)              0         
                                                                 
 reshape_8 (Reshape)         (None, 7, 7, 128)         0         
                                                                 
 up_sampling2d_4 (UpSamplin  (None, 14, 14, 128)       0         
 g2D)                                                            
                                                                 
 leaky_re_lu_23 (LeakyReLU)  (None, 14, 14, 128)       0         
                                                                 
 up_sampling2d_5 (UpSamplin  (None, 28, 28, 128)       0         
 g2D)                                                 

In [12]:
def define_discriminator():
    model = Sequential()

    model.add(Conv2D(64, (3, 3), input_shape=(28, 28, 2), padding='same'))
    model.add(LeakyReLU(0.2))

    model.add(Conv2D(128, (3, 3), strides= (2, 2), padding='same'))
    model.add(LeakyReLU(0.2))
    
    model.add(Conv2D(128, (3, 3), strides= (2, 2), padding='same'))
    model.add(LeakyReLU(0.2))

    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(1, activation='sigmoid'))

    img = Input(shape=(28, 28, 1))
    label = Input(shape=(1,))
    
    label_embed = Embedding(input_dim=10, output_dim=np.prod((28, 28)), input_length=1)(label)
    label_embed = Flatten()(label_embed)
    label_embed = Reshape((28, 28, 1))(label_embed)
    
    concatenated = Concatenate(axis=-1)([label_embed, img])
    
    prediction = model(concatenated)
    d_model =Model([img, label], prediction)
    d_model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='adam')


    return d_model,model

# Example usage
d_model,model = define_discriminator()
model.summary()


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_13 (Conv2D)          (None, 28, 28, 64)        1216      
                                                                 
 leaky_re_lu_13 (LeakyReLU)  (None, 28, 28, 64)        0         
                                                                 
 conv2d_14 (Conv2D)          (None, 14, 14, 128)       73856     
                                                                 
 leaky_re_lu_14 (LeakyReLU)  (None, 14, 14, 128)       0         
                                                                 
 conv2d_15 (Conv2D)          (None, 7, 7, 128)         147584    
                                                                 
 leaky_re_lu_15 (LeakyReLU)  (None, 7, 7, 128)         0         
                                                                 
 flatten_10 (Flatten)        (None, 6272)             

Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [97]:
def define_gan(g_model, d_model):
    
	d_model.trainable = False
	model=Sequential()
	model.add(g_model)
	model.add(d_model)
	opt = Adam(learning_rate=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt)
	return model


In [21]:
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()


In [98]:

def load_real_samples():
	(x_train, y_train), (_, _) =  tf.keras.datasets.mnist.load_data()
	X = X.astype('float32')
	X = (X - 127.5) / 127.5 #-1,1
	return [X, y_train]

def generate_real_samples(dataset, n_samples):
	images, labels = dataset
	ix = np.random.randint(0, images.shape[0], n_samples)
	X, labels = images[ix], labels[ix]
	y = np.ones((n_samples, 1))
	return [X, labels], y    #x is images and y is 1 that is real


In [99]:

def generate_latent_points(latent_dim, n_samples, n_classes=10):
	x_input = np.random.randn(latent_dim * n_samples)
	z_input = x_input.reshape(n_samples, latent_dim)
	labels = np.random.randint(0, n_classes, n_samples)
	return [z_input, labels]

def generate_fake_samples(generator, latent_dim, n_samples):
	z_input, labels_input = generate_latent_points(latent_dim, n_samples)
	images = generator.predict([z_input, labels_input])
	y = np.zeros((n_samples, 1))
	return [images, labels_input], y   #x is images and y is 0 that is fake


In [113]:
from keras.models import load_model

def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=30, n_batch=128):
	bat_per_epo = int(dataset[0].shape[0] / n_batch)
	half_batch = int(n_batch / 2)
	for i in range(n_epochs):
		for j in range(bat_per_epo):
      
			[X_real, labels_real], y_real = generate_real_samples(dataset, half_batch)
			d_loss1, _ = d_model.train_on_batch([X_real, labels_real], y_real)
   
			[X_fake, labels], y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
			d_loss2, _ = d_model.train_on_batch([X_fake, labels], y_fake)
   
			[z_input, labels_input] = generate_latent_points(latent_dim, n_batch)
			y_gan = np.ones((n_batch, 1))
			g_loss = gan_model.train_on_batch([z_input, labels_input], y_gan)

			print('>%d, %d/%d, d1=%.3f, d2=%.3f g=%.3f' %
				(i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))
	g_model.save('CGANsgenerator.h5')

latent_dim = 100
d_model = define_discriminator()
g_model = load_model('CGANs_generator.h5')
gan_model = define_gan(g_model, d_model)
dataset = load_real_samples()
train(g_model, d_model, gan_model, dataset, latent_dim)

2/2 [==============================] - 1s 478ms/step
>1, 1/468, d1=1.276, d2=13.265 g=0.637
2/2 [==============================] - 0s 216ms/step
>1, 2/468, d1=0.000, d2=1.673 g=0.913
2/2 [==============================] - 0s 148ms/step
>1, 3/468, d1=0.520, d2=0.009 g=1.126
2/2 [==============================] - 0s 162ms/step
>1, 4/468, d1=2.467, d2=0.002 g=0.997
2/2 [==============================] - 1s 240ms/step
>1, 5/468, d1=0.593, d2=0.075 g=0.862
2/2 [==============================] - 0s 274ms/step
>1, 6/468, d1=0.111, d2=0.390 g=0.766
2/2 [==============================] - 0s 134ms/step
>1, 7/468, d1=0.026, d2=0.812 g=0.742
2/2 [==============================] - 0s 213ms/step
>1, 8/468, d1=0.058, d2=0.474 g=0.777
2/2 [==============================] - 0s 211ms/step
>1, 9/468, d1=0.035, d2=0.153 g=0.825
2/2 [==============================] - 0s 229ms/step
>1, 10/468, d1=0.205, d2=0.050 g=0.866
2/2 [==============================] - 0s 228ms/step
>1, 11/468, d1=0.246, d2=0.069 g=0.

In [1]:
def generate_latent_points(latent_dim, n_samples, label):
    z_input = np.random.normal(0, 1, size=(n_samples, latent_dim))
    labels_input = np.full((n_samples, 1), label)
    return z_input, labels_input

def generate_and_plot_sample(generator, latent_dim, label):
    z_input, labels_input = generate_latent_points(latent_dim, 1, label)
    generated_sample = generator.predict([z_input, labels_input])[0, :, :, 0]

    plt.imshow(generated_sample, cmap='gray')
    plt.title(f'Generated Image for Label {label}')
    plt.axis('off')
    plt.show()

latent_dim = 100
g_model = load_model('CGANs_generator.h5')

user_label = int(input("Enter any digit "))

generate_and_plot_sample(g_model, latent_dim, user_label)


NameError: name 'load_model' is not defined